In [9]:
# pip install Selenium
# pip install webdriver-manager

In [10]:
import pandas as pd
import json
import csv
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

In [11]:
file_name = "nba.csv"

In [12]:
chrome_options = Options()
# chrome_options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=chrome_options)

In [13]:
years = ['2020-21','2021-22','2022-23']

In [14]:
def generate_json(data, index):    
    with open(f"products_{index}.json", "w") as outfile:
        json.dump(data, outfile,indent = 4)

In [15]:
def scrape_data():
    rows = []
    for year in years:
        url = f"https://www.nba.com/stats/leaders?Season={year}"
        driver.get(url)
        pagination_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.Crom_cromSettings__ak6Hd div.DropDown_dropdown__TMlAR > select")))
        pagination_select = Select(pagination_element)
        print("select",len(pagination_select.options))
        pagination_select.select_by_index(0)
        pagination_select.select_by_visible_text("All")

        nba_player_table_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.Crom_container__C45Ti.crom-container > table")))


        headers = []
        table_head_element = nba_player_table_element.find_elements(By.CSS_SELECTOR, "thead > tr th")
        for th in table_head_element:
            headers.append(th.text)
        print("header",headers)
        headers.append("SEASON")

        table_body_element = nba_player_table_element.find_elements(By.CSS_SELECTOR, "tbody > tr")
        print("table_body_element",table_body_element)
        with open(file_name, "a") as file:
            for tr in table_body_element:
                tds = tr.find_elements(By.CSS_SELECTOR, "tr > td")
                data = {
                    "SEASON": year
                }
                row = []
                for td, idx in zip(tds,range(len(tds))):
                    data[headers[idx]] = td.text
                    row.append(td.text)
                print(idx, data)
                writer = csv.DictWriter(file, fieldnames=headers)
                row.append(year)
                writer.writerow(data)
                rows.append(row)

        print(headers)
        print(rows)
    return {
        "headers":headers,
        "rows":rows,
    }

In [16]:
result = scrape_data() 

select 6
header ['#', 'PLAYER', 'TEAM', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'EFF']
table_body_element [<selenium.webdriver.remote.webelement.WebElement (session="93bdf007c9a09caa33c42d1ec4ef5660", element="C60138D7A87E5B5367C2D70BD08EFE24_element_654")>, <selenium.webdriver.remote.webelement.WebElement (session="93bdf007c9a09caa33c42d1ec4ef5660", element="C60138D7A87E5B5367C2D70BD08EFE24_element_714")>, <selenium.webdriver.remote.webelement.WebElement (session="93bdf007c9a09caa33c42d1ec4ef5660", element="C60138D7A87E5B5367C2D70BD08EFE24_element_774")>, <selenium.webdriver.remote.webelement.WebElement (session="93bdf007c9a09caa33c42d1ec4ef5660", element="C60138D7A87E5B5367C2D70BD08EFE24_element_834")>, <selenium.webdriver.remote.webelement.WebElement (session="93bdf007c9a09caa33c42d1ec4ef5660", element="C60138D7A87E5B5367C2D70BD08EFE24_element_894")>, <selenium.webdriver.remote.webelement.W

In [18]:
headers = result["headers"]
rows = result["rows"]
df = pd.DataFrame(rows, columns=headers, dtype = float)
df

c:\Users\aksha\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)


,#,PLAYER,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,SEASON
0,1.0,Stephen Curry,GSW,63.0,34.2,32.0,10.4,21.7,48.2,5.3,...,91.6,0.5,5.0,5.5,5.8,1.2,0.1,3.4,29.4,2020-21
1,2.0,Bradley Beal,WAS,60.0,35.8,31.3,11.2,23.0,48.5,2.2,...,88.9,1.2,3.5,4.7,4.4,1.2,0.4,3.1,26.1,2020-21
2,3.0,Damian Lillard,POR,67.0,35.8,28.8,9.0,19.9,45.1,4.1,...,92.8,0.5,3.7,4.2,7.5,0.9,0.3,3.0,27.2,2020-21
3,4.0,Joel Embiid,PHI,51.0,31.1,28.5,9.0,17.6,51.3,1.1,...,85.9,2.2,8.4,10.6,2.8,1.0,1.4,3.1,31.0,2020-21
4,5.0,Giannis Antetokounmpo,MIL,61.0,33.0,28.1,10.3,18.0,56.9,1.1,...,68.5,1.6,9.4,11.0,5.9,1.2,1.2,3.4,33.2,2020-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,241.0,P.J. Tucker,PHI,75.0,25.6,3.5,1.3,3.0,42.7,0.7,...,82.6,1.3,2.7,3.9,0.8,0.5,0.2,0.6,6.6,2022-23
706,242.0,Miles McBride,NYK,64.0,11.9,3.5,1.2,3.4,35.8,0.6,...,66.7,0.2,0.6,0.8,1.1,0.6,0.1,0.4,3.3,2022-23
707,243.0,Anthony Gill,WAS,59.0,10.6,3.3,1.2,2.2,53.8,0.1,...,73.1,0.6,1.1,1.7,0.6,0.1,0.2,0.3,4.3,2022-23
708,244.0,Christian Koloko,TOR,58.0,13.8,3.1,1.2,2.6,48.0,0.0,...,62.7,1.4,1.5,2.9,0.5,0.4,1.0,0.3,5.9,2022-23


In [19]:
df

,#,PLAYER,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,SEASON
0,1.0,Stephen Curry,GSW,63.0,34.2,32.0,10.4,21.7,48.2,5.3,...,91.6,0.5,5.0,5.5,5.8,1.2,0.1,3.4,29.4,2020-21
1,2.0,Bradley Beal,WAS,60.0,35.8,31.3,11.2,23.0,48.5,2.2,...,88.9,1.2,3.5,4.7,4.4,1.2,0.4,3.1,26.1,2020-21
2,3.0,Damian Lillard,POR,67.0,35.8,28.8,9.0,19.9,45.1,4.1,...,92.8,0.5,3.7,4.2,7.5,0.9,0.3,3.0,27.2,2020-21
3,4.0,Joel Embiid,PHI,51.0,31.1,28.5,9.0,17.6,51.3,1.1,...,85.9,2.2,8.4,10.6,2.8,1.0,1.4,3.1,31.0,2020-21
4,5.0,Giannis Antetokounmpo,MIL,61.0,33.0,28.1,10.3,18.0,56.9,1.1,...,68.5,1.6,9.4,11.0,5.9,1.2,1.2,3.4,33.2,2020-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,241.0,P.J. Tucker,PHI,75.0,25.6,3.5,1.3,3.0,42.7,0.7,...,82.6,1.3,2.7,3.9,0.8,0.5,0.2,0.6,6.6,2022-23
706,242.0,Miles McBride,NYK,64.0,11.9,3.5,1.2,3.4,35.8,0.6,...,66.7,0.2,0.6,0.8,1.1,0.6,0.1,0.4,3.3,2022-23
707,243.0,Anthony Gill,WAS,59.0,10.6,3.3,1.2,2.2,53.8,0.1,...,73.1,0.6,1.1,1.7,0.6,0.1,0.2,0.3,4.3,2022-23
708,244.0,Christian Koloko,TOR,58.0,13.8,3.1,1.2,2.6,48.0,0.0,...,62.7,1.4,1.5,2.9,0.5,0.4,1.0,0.3,5.9,2022-23


In [21]:
df.to_csv("nba-full.csv",index=False)